In [1]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers.core import Activation
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import UpSampling2D
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.layers.core import Flatten
from keras.optimizers import SGD
from keras.layers import GaussianNoise
import keras.backend as K
import numpy as np
from PIL import Image
import math
import matplotlib.pyplot as plt
import os

Using TensorFlow backend.


In [3]:
X_train = np.load('preprocessed/x_train_prep.npy')

In [4]:
X_train = X_train[:,:,:,1]
X_train = (X_train.astype(np.float32) - 127.5)/127.5
X_train = X_train.reshape((X_train.shape[0], ) + X_train.shape[1:] + (1,))
X_train.shape

(3000, 224, 224, 1)

In [5]:
# for reproducibility
np.random.seed(3768)

# use this environment flag to change which GPU to use 
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"  # specify which GPU(s) to be used

#Get TensorFlow session
def get_session(): 
  config = tf.ConfigProto() 
  config.gpu_options.allow_growth = True 
  return tf.Session(config=config) 

In [6]:
def discriminator_model():
    model = Sequential()
    input_shape = (224,224,1)
    model.add(Convolution2D(8, (5, 5), padding="same", input_shape=input_shape))
    model.add(Activation('tanh'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Convolution2D(16, (5, 5)))
    model.add(Activation('tanh'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Convolution2D(32, (5, 5)))
    model.add(Activation('tanh'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Convolution2D(64, (5, 5)))
    model.add(Activation('tanh'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Convolution2D(128, (5, 5)))
    model.add(Activation('tanh'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Flatten())
    
    model.add(Dense(528))
    model.add(Activation('tanh'))
    
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    return model

In [7]:
def generator_model():
    model = Sequential()
    model.add(Dense(input_dim=100, units=512))
    model.add(Activation('tanh'))
    
    model.add(Dense(128*7*7))
    model.add(BatchNormalization())
    model.add(Activation('tanh'))
    
    model.add(Reshape((7, 7, 128), input_shape=(128*7*7,)))
    model.add(UpSampling2D(size=(2, 2)))
    
    model.add(Convolution2D(56, (5, 5), padding='same'))
    model.add(Activation('tanh'))
    model.add(UpSampling2D(size=(2, 2)))
    
    model.add(Convolution2D(28, (5, 5), padding='same'))
    model.add(Activation('tanh'))
    model.add(UpSampling2D(size=(2, 2)))
    
    model.add(Convolution2D(14, (5, 5), padding='same'))
    model.add(Activation('tanh'))
    model.add(UpSampling2D(size=(2, 2)))
    
    model.add(Convolution2D(7, (5, 5), padding='same'))
    model.add(Activation('tanh'))
    model.add(UpSampling2D(size=(2, 2)))
    
    model.add(Convolution2D(1, (5, 5), padding='same'))
    model.add(Activation('tanh'))
    return model

In [10]:
def generator_containing_discriminator(generator, discriminator):
    model = Sequential()
    model.add(generator)
    discriminator.trainable = False
    model.add(discriminator)
    return model

In [11]:
def combine_images(generated_images):
    num = generated_images.shape[0]
    width = int(math.sqrt(num))
    height = int(math.ceil(float(num)/width))
    shape = generated_images.shape[1:3]

    image = np.zeros((height*shape[0], width*shape[1]),
                     dtype=generated_images.dtype)
    for index, img in enumerate(generated_images):
        i = int(index/width)
        j = index % width
        image[i*shape[0]:(i+1)*shape[0], j*shape[1]:(j+1)*shape[1]] = img[:, :, 0]
    return image

In [12]:
def train(batch_size, epoch_num, X_train):
    discriminator = discriminator_model()
    generator = generator_model()
    discriminator_on_generator = generator_containing_discriminator(generator, discriminator)
    d_optim = SGD(lr=0.0001, momentum=0.8, nesterov=True)
    g_optim = SGD(lr=0.0001, momentum=0.8, nesterov=True)
    generator.compile(loss='binary_crossentropy', optimizer="SGD")
    discriminator_on_generator.compile(
        loss='binary_crossentropy', optimizer=g_optim)
    discriminator.trainable = True
    discriminator.compile(loss='binary_crossentropy', optimizer=d_optim)
    noise = np.zeros((batch_size, 100))

    false_negative_list = []
    false_positive_list = []
    total_accuracy_list = []
    for epoch in range(epoch_num):
        print('~'*70)
        print("Epoch is", epoch)
        print("Number of batches", int(X_train.shape[0]/batch_size))
        batches_num = int(X_train.shape[0]/batch_size)
        # we train the generator first
        for index in range(batches_num):

            for i in range(batch_size):
                noise[i, :] = np.random.uniform(-1, 1, 100)
            # note that when train the generator, the discriminator is not trainalbe
            discriminator.trainable = False
            g_loss = discriminator_on_generator.train_on_batch(
                noise, [1] * batch_size)
            discriminator.trainable = True
            print("epoch %d/%d batch %d/%d g_loss : %f" % (epoch+1, epoch_num,index, batches_num, g_loss))

            # leverage the generative images and natural images to train the discriminator
            for i in range(batch_size):
                noise[i, :] = np.random.uniform(-1, 1, 100)
            image_batch = X_train[index*batch_size:(index+1)*batch_size]
            generated_images = generator.predict(noise, verbose=0)
            # save the generative images on the process of training regularly
            if index % 80 == 0:
                image = combine_images(generated_images)
                image = image*127.5+127.5
                Image.fromarray(image.astype(np.uint8)).save('images/'+
                    str(epoch)+"_"+str(index)+".png")
            # create the trainset of the discriminator
            X = np.concatenate((image_batch, generated_images))
            y = [1] * batch_size + [0] * batch_size
            d_loss = discriminator.train_on_batch(X, y)
            print("epoch %d/%d batch %d/%d d_loss : %f" % (epoch+1, epoch_num, index, batches_num, d_loss))

            # calculate the accuracy of the generator and discriminator
            if  index%20 == 0:
                # to test the generator  and discriminator's ability
                y_pred = discriminator.predict(X)
                false_positive = float(np.sum(y_pred[batch_size:]>0.5))/batch_size
                true_negative = float(np.sum(y_pred[batch_size:]<0.5))/batch_size
                true_positive = float(np.sum(y_pred[:batch_size]>0.5))/batch_size
                false_negative = float(np.sum(y_pred[:batch_size]<0.5))/batch_size
                total_accuracy = (true_positive + true_negative) /2

                print('*'*10)
                print("the ratio that generator deceive discrimator successfully is :%f"%false_positive)
                print("the ratio that discrimator fail to discrimate true/natural images:%f"%false_negative)
                print("the total accuracy of the discriminator is :%f"%total_accuracy)

                false_negative_list.append(false_negative)
                false_positive_list.append(false_positive)
                total_accuracy_list.append(total_accuracy)

            if index % 20 == 0:
                generator.save_weights('generator', True)
                discriminator.save_weights('discriminator', True)

In [20]:
def generate(batch_size, nice=False):
    generator = generator_model()
    generator.compile(loss='binary_crossentropy', optimizer="SGD")
    generator.load_weights('generator')
    # if nice is true, choose the 5% best generative images for storage, else the original generative images
    if nice:
        discriminator = discriminator_model()
        discriminator.compile(loss='binary_crossentropy', optimizer="SGD")
        discriminator.load_weights('discriminator')
        noise = np.zeros((batch_size*40, 100))
        for i in range(batch_size*40):
            noise[i, :] = np.random.uniform(-1, 1, 100)
        generated_images = generator.predict(noise, verbose=1)
        d_pret = discriminator.predict(generated_images, verbose=1)
        index = np.arange(0, batch_size*40)
        index.resize((batch_size*40, 1))
        pre_with_index = list(np.append(d_pret, index, axis=1))
        pre_with_index.sort(key=lambda x: x[0], reverse=True)
        nice_images = np.zeros((batch_size, ) +
                    (generated_images.shape[1:3]) + (1,), dtype=np.float32)
        for i in range(int(batch_size)):
            idx = int(pre_with_index[i][1])
            nice_images[i, :, :, 0] = generated_images[idx, :, :, 0]
        image = combine_images(nice_images)
    else:
        noise = np.zeros((batch_size, 100))
        for i in range(batch_size):
            noise[i, :] = np.random.uniform(-1, 1, 100)
        generated_images = generator.predict(noise, verbose=1)
        image = combine_images(generated_images)
    image = image*127.5+127.5
    Image.fromarray(image.astype(np.uint8)).save(
        "generated_image.png")

In [14]:
 train(batch_size=100 , epoch_num=400, X_train=X_train)

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Epoch is 0
Number of batches 30
epoch 1/400 batch 0/30 g_loss : 0.528875
epoch 1/400 batch 0/30 d_loss : 0.704078
**********
the ratio that generator deceive discrimator successfully is :1.000000
the ratio that discrimator fail to discrimate true/natural images:0.000000
the total accuracy of the discriminator is :0.500000
epoch 1/400 batch 1/30 g_loss : 0.529748
epoch 1/400 batch 1/30 d_loss : 0.706316
epoch 1/400 batch 2/30 g_loss : 0.531585
epoch 1/400 batch 2/30 d_loss : 0.704639
epoch 1/400 batch 3/30 g_loss : 0.529954
epoch 1/400 batch 3/30 d_loss : 0.706765
epoch 1/400 batch 4/30 g_loss : 0.532031
epoch 1/400 batch 4/30 d_loss : 0.703142
epoch 1/400 batch 5/30 g_loss : 0.530325
epoch 1/400 batch 5/30 d_loss : 0.703491
epoch 1/400 batch 6/30 g_loss : 0.534938
epoch 1/400 batch 6/30 d_loss : 0.703363
epoch 1/400 batch 7/30 g_loss : 0.532643
epoch 1/400 batch 7/30 d_loss : 0.701097
epoch 1/400 batch 8/30 g_loss :

epoch 3/400 batch 21/30 g_loss : 0.617500
epoch 3/400 batch 21/30 d_loss : 0.617363
epoch 3/400 batch 22/30 g_loss : 0.622146
epoch 3/400 batch 22/30 d_loss : 0.615693
epoch 3/400 batch 23/30 g_loss : 0.619932
epoch 3/400 batch 23/30 d_loss : 0.624767
epoch 3/400 batch 24/30 g_loss : 0.630910
epoch 3/400 batch 24/30 d_loss : 0.612283
epoch 3/400 batch 25/30 g_loss : 0.625224
epoch 3/400 batch 25/30 d_loss : 0.615904
epoch 3/400 batch 26/30 g_loss : 0.628100
epoch 3/400 batch 26/30 d_loss : 0.618251
epoch 3/400 batch 27/30 g_loss : 0.637733
epoch 3/400 batch 27/30 d_loss : 0.610484
epoch 3/400 batch 28/30 g_loss : 0.632735
epoch 3/400 batch 28/30 d_loss : 0.610390
epoch 3/400 batch 29/30 g_loss : 0.633302
epoch 3/400 batch 29/30 d_loss : 0.604665
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Epoch is 3
Number of batches 30
epoch 4/400 batch 0/30 g_loss : 0.632336
epoch 4/400 batch 0/30 d_loss : 0.601901
**********
the ratio that generator deceive discrimator suc

epoch 6/400 batch 13/30 d_loss : 0.482923
epoch 6/400 batch 14/30 g_loss : 0.800954
epoch 6/400 batch 14/30 d_loss : 0.482576
epoch 6/400 batch 15/30 g_loss : 0.798482
epoch 6/400 batch 15/30 d_loss : 0.484068
epoch 6/400 batch 16/30 g_loss : 0.800638
epoch 6/400 batch 16/30 d_loss : 0.485304
epoch 6/400 batch 17/30 g_loss : 0.803399
epoch 6/400 batch 17/30 d_loss : 0.476955
epoch 6/400 batch 18/30 g_loss : 0.822087
epoch 6/400 batch 18/30 d_loss : 0.481141
epoch 6/400 batch 19/30 g_loss : 0.813042
epoch 6/400 batch 19/30 d_loss : 0.476120
epoch 6/400 batch 20/30 g_loss : 0.814725
epoch 6/400 batch 20/30 d_loss : 0.487362
**********
the ratio that generator deceive discrimator successfully is :0.190000
the ratio that discrimator fail to discrimate true/natural images:0.000000
the total accuracy of the discriminator is :0.905000
epoch 6/400 batch 21/30 g_loss : 0.827223
epoch 6/400 batch 21/30 d_loss : 0.491879
epoch 6/400 batch 22/30 g_loss : 0.812788
epoch 6/400 batch 22/30 d_loss : 0

epoch 9/400 batch 3/30 d_loss : 0.379908
epoch 9/400 batch 4/30 g_loss : 1.078280
epoch 9/400 batch 4/30 d_loss : 0.387948
epoch 9/400 batch 5/30 g_loss : 1.063512
epoch 9/400 batch 5/30 d_loss : 0.356267
epoch 9/400 batch 6/30 g_loss : 1.071265
epoch 9/400 batch 6/30 d_loss : 0.370813
epoch 9/400 batch 7/30 g_loss : 1.082171
epoch 9/400 batch 7/30 d_loss : 0.365453
epoch 9/400 batch 8/30 g_loss : 1.089945
epoch 9/400 batch 8/30 d_loss : 0.364988
epoch 9/400 batch 9/30 g_loss : 1.065972
epoch 9/400 batch 9/30 d_loss : 0.358652
epoch 9/400 batch 10/30 g_loss : 1.078618
epoch 9/400 batch 10/30 d_loss : 0.349261
epoch 9/400 batch 11/30 g_loss : 1.096055
epoch 9/400 batch 11/30 d_loss : 0.354424
epoch 9/400 batch 12/30 g_loss : 1.081879
epoch 9/400 batch 12/30 d_loss : 0.349028
epoch 9/400 batch 13/30 g_loss : 1.088989
epoch 9/400 batch 13/30 d_loss : 0.332047
epoch 9/400 batch 14/30 g_loss : 1.101208
epoch 9/400 batch 14/30 d_loss : 0.346370
epoch 9/400 batch 15/30 g_loss : 1.102891
epoch

epoch 11/400 batch 25/30 d_loss : 0.298658
epoch 11/400 batch 26/30 g_loss : 1.401767
epoch 11/400 batch 26/30 d_loss : 0.289515
epoch 11/400 batch 27/30 g_loss : 1.401358
epoch 11/400 batch 27/30 d_loss : 0.277681
epoch 11/400 batch 28/30 g_loss : 1.382989
epoch 11/400 batch 28/30 d_loss : 0.265327
epoch 11/400 batch 29/30 g_loss : 1.388318
epoch 11/400 batch 29/30 d_loss : 0.266600
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Epoch is 11
Number of batches 30
epoch 12/400 batch 0/30 g_loss : 1.389539
epoch 12/400 batch 0/30 d_loss : 0.276421
**********
the ratio that generator deceive discrimator successfully is :0.030000
the ratio that discrimator fail to discrimate true/natural images:0.030000
the total accuracy of the discriminator is :0.970000
epoch 12/400 batch 1/30 g_loss : 1.413207
epoch 12/400 batch 1/30 d_loss : 0.275323
epoch 12/400 batch 2/30 g_loss : 1.420098
epoch 12/400 batch 2/30 d_loss : 0.278082
epoch 12/400 batch 3/30 g_loss : 1.419299
epoch

...

epoch 388/400 batch 24/30 d_loss : 0.598177
epoch 388/400 batch 25/30 g_loss : 1.143860
epoch 388/400 batch 25/30 d_loss : 0.583851
epoch 388/400 batch 26/30 g_loss : 1.228085
epoch 388/400 batch 26/30 d_loss : 0.569473
epoch 388/400 batch 27/30 g_loss : 1.171927
epoch 388/400 batch 27/30 d_loss : 0.562620
epoch 388/400 batch 28/30 g_loss : 1.085370
epoch 388/400 batch 28/30 d_loss : 0.556998
epoch 388/400 batch 29/30 g_loss : 1.025500
epoch 388/400 batch 29/30 d_loss : 0.575410
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Epoch is 388
Number of batches 30
epoch 389/400 batch 0/30 g_loss : 0.987020
epoch 389/400 batch 0/30 d_loss : 0.570327
**********
the ratio that generator deceive discrimator successfully is :0.200000
the ratio that discrimator fail to discrimate true/natural images:0.270000
the total accuracy of the discriminator is :0.765000
epoch 389/400 batch 1/30 g_loss : 0.893013
epoch 389/400 batch 1/30 d_loss : 0.579160
epoch 389/400 batch 2/30 g_lo

epoch 391/400 batch 13/30 d_loss : 0.639079
epoch 391/400 batch 14/30 g_loss : 0.787393
epoch 391/400 batch 14/30 d_loss : 0.634083
epoch 391/400 batch 15/30 g_loss : 0.709090
epoch 391/400 batch 15/30 d_loss : 0.655455
epoch 391/400 batch 16/30 g_loss : 0.743998
epoch 391/400 batch 16/30 d_loss : 0.655961
epoch 391/400 batch 17/30 g_loss : 0.777682
epoch 391/400 batch 17/30 d_loss : 0.690529
epoch 391/400 batch 18/30 g_loss : 0.766454
epoch 391/400 batch 18/30 d_loss : 0.690431
epoch 391/400 batch 19/30 g_loss : 0.793489
epoch 391/400 batch 19/30 d_loss : 0.667142
epoch 391/400 batch 20/30 g_loss : 0.846301
epoch 391/400 batch 20/30 d_loss : 0.669688
**********
the ratio that generator deceive discrimator successfully is :0.210000
the ratio that discrimator fail to discrimate true/natural images:0.490000
the total accuracy of the discriminator is :0.650000
epoch 391/400 batch 21/30 g_loss : 0.853452
epoch 391/400 batch 21/30 d_loss : 0.655208
epoch 391/400 batch 22/30 g_loss : 0.98594

epoch 394/400 batch 1/30 g_loss : 0.901362
epoch 394/400 batch 1/30 d_loss : 0.558274
epoch 394/400 batch 2/30 g_loss : 0.806958
epoch 394/400 batch 2/30 d_loss : 0.607798
epoch 394/400 batch 3/30 g_loss : 0.835004
epoch 394/400 batch 3/30 d_loss : 0.570121
epoch 394/400 batch 4/30 g_loss : 0.804302
epoch 394/400 batch 4/30 d_loss : 0.568843
epoch 394/400 batch 5/30 g_loss : 0.844652
epoch 394/400 batch 5/30 d_loss : 0.532897
epoch 394/400 batch 6/30 g_loss : 0.862115
epoch 394/400 batch 6/30 d_loss : 0.528374
epoch 394/400 batch 7/30 g_loss : 0.865310
epoch 394/400 batch 7/30 d_loss : 0.502805
epoch 394/400 batch 8/30 g_loss : 0.919095
epoch 394/400 batch 8/30 d_loss : 0.500480
epoch 394/400 batch 9/30 g_loss : 0.906148
epoch 394/400 batch 9/30 d_loss : 0.465417
epoch 394/400 batch 10/30 g_loss : 0.924468
epoch 394/400 batch 10/30 d_loss : 0.587412
epoch 394/400 batch 11/30 g_loss : 0.883298
epoch 394/400 batch 11/30 d_loss : 0.585734
epoch 394/400 batch 12/30 g_loss : 0.848961
epoch 

epoch 396/400 batch 21/30 g_loss : 0.867254
epoch 396/400 batch 21/30 d_loss : 0.648886
epoch 396/400 batch 22/30 g_loss : 0.992584
epoch 396/400 batch 22/30 d_loss : 0.623677
epoch 396/400 batch 23/30 g_loss : 0.983285
epoch 396/400 batch 23/30 d_loss : 0.585488
epoch 396/400 batch 24/30 g_loss : 1.052328
epoch 396/400 batch 24/30 d_loss : 0.594384
epoch 396/400 batch 25/30 g_loss : 1.120880
epoch 396/400 batch 25/30 d_loss : 0.594772
epoch 396/400 batch 26/30 g_loss : 1.103323
epoch 396/400 batch 26/30 d_loss : 0.564543
epoch 396/400 batch 27/30 g_loss : 1.101088
epoch 396/400 batch 27/30 d_loss : 0.556063
epoch 396/400 batch 28/30 g_loss : 1.054557
epoch 396/400 batch 28/30 d_loss : 0.565023
epoch 396/400 batch 29/30 g_loss : 0.959308
epoch 396/400 batch 29/30 d_loss : 0.559326
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Epoch is 396
Number of batches 30
epoch 397/400 batch 0/30 g_loss : 0.928233
epoch 397/400 batch 0/30 d_loss : 0.576668
**********
the ra

epoch 399/400 batch 9/30 d_loss : 0.472251
epoch 399/400 batch 10/30 g_loss : 0.906573
epoch 399/400 batch 10/30 d_loss : 0.591841
epoch 399/400 batch 11/30 g_loss : 0.889464
epoch 399/400 batch 11/30 d_loss : 0.586547
epoch 399/400 batch 12/30 g_loss : 0.832253
epoch 399/400 batch 12/30 d_loss : 0.612963
epoch 399/400 batch 13/30 g_loss : 0.822301
epoch 399/400 batch 13/30 d_loss : 0.648102
epoch 399/400 batch 14/30 g_loss : 0.738689
epoch 399/400 batch 14/30 d_loss : 0.622123
epoch 399/400 batch 15/30 g_loss : 0.771631
epoch 399/400 batch 15/30 d_loss : 0.645200
epoch 399/400 batch 16/30 g_loss : 0.748626
epoch 399/400 batch 16/30 d_loss : 0.654799
epoch 399/400 batch 17/30 g_loss : 0.733525
epoch 399/400 batch 17/30 d_loss : 0.673378
epoch 399/400 batch 18/30 g_loss : 0.773191
epoch 399/400 batch 18/30 d_loss : 0.694470
epoch 399/400 batch 19/30 g_loss : 0.767254
epoch 399/400 batch 19/30 d_loss : 0.636883
epoch 399/400 batch 20/30 g_loss : 0.808009
epoch 399/400 batch 20/30 d_loss 

In [27]:
generate(batch_size=100, nice=True)

4000/4000 [==============================] - 2s 510us/step


In [16]:
d = discriminator_model()

In [17]:
g = generator_model()

In [18]:
for layer in d.layers:
    print("i",layer.input_shape)
    print("o",layer.output_shape)

i (None, 224, 224, 1)
o (None, 224, 224, 8)
i (None, 224, 224, 8)
o (None, 224, 224, 8)
i (None, 224, 224, 8)
o (None, 112, 112, 8)
i (None, 112, 112, 8)
o (None, 108, 108, 16)
i (None, 108, 108, 16)
o (None, 108, 108, 16)
i (None, 108, 108, 16)
o (None, 54, 54, 16)
i (None, 54, 54, 16)
o (None, 50, 50, 32)
i (None, 50, 50, 32)
o (None, 50, 50, 32)
i (None, 50, 50, 32)
o (None, 25, 25, 32)
i (None, 25, 25, 32)
o (None, 21, 21, 64)
i (None, 21, 21, 64)
o (None, 21, 21, 64)
i (None, 21, 21, 64)
o (None, 10, 10, 64)
i (None, 10, 10, 64)
o (None, 6, 6, 128)
i (None, 6, 6, 128)
o (None, 6, 6, 128)
i (None, 6, 6, 128)
o (None, 3, 3, 128)
i (None, 3, 3, 128)
o (None, 1152)
i (None, 1152)
o (None, 528)
i (None, 528)
o (None, 528)
i (None, 528)
o (None, 1)
i (None, 1)
o (None, 1)


In [19]:
for layer in g.layers:
    print("i",layer.input_shape)
    print("o",layer.output_shape)

o (None, 512)
o (None, 512)
o (None, 6272)
o (None, 6272)
o (None, 6272)
o (None, 7, 7, 128)
o (None, 14, 14, 128)
o (None, 14, 14, 56)
o (None, 14, 14, 56)
o (None, 28, 28, 56)
o (None, 28, 28, 28)
o (None, 28, 28, 28)
o (None, 56, 56, 28)
o (None, 56, 56, 14)
o (None, 56, 56, 14)
o (None, 112, 112, 14)
o (None, 112, 112, 7)
o (None, 112, 112, 7)
o (None, 224, 224, 7)
o (None, 224, 224, 1)
o (None, 224, 224, 1)
